In [48]:
import PIL
import os
import tensorflow as tf
import numpy as np
import pandas
from skimage.io import imread
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [21]:
directory_path = 'data/images/training_data'

In [22]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory_path,
    image_size=(256, 256),  # Resize images to 256x256
    batch_size=32,  # Number of images to return in each batch
    label_mode='categorical',  # Can be 'int', 'categorical', or 'binary'
    validation_split=0.2,
    subset='training',
    seed=42
)

Found 3164 files belonging to 3 classes.
Using 2532 files for training.


In [23]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory_path,
    image_size=(256, 256),  # Resize images to 256x256
    batch_size=32,  # Number of images to return in each batch
    label_mode='categorical',  # Can be 'int', 'categorical', or 'binary'
    validation_split=0.2,
    subset='validation',
    seed=42
)

Found 3164 files belonging to 3 classes.
Using 632 files for validation.


In [24]:
train_ds.class_names

['cringe', 'funny', 'neutral']

In [25]:
tf.__version__

'2.17.0'

In [41]:
resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(256,256,3),
    pooling='avg',
    classes=3
)
 
for layer in pretrained_model.layers:
    layer.trainable = False
    
resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(512, activation='relu'))
resnet_model.add(layers.Dense(3, activation='softmax'))

In [42]:
resnet_model.summary()

ValueError: Undefined shapes are not supported.

This model does not compile on my machine. ChatGPT spit out the code for the model below

In [49]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model

# Define the input tensor
input_tensor = Input(shape=(256, 256, 3))

# Load the pre-trained ResNet50 model without the top layer
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    pooling='avg',
    classes=3,
    input_tensor=input_tensor
)

# Add a Flatten layer to convert the 4D output to 2D
flatten = layers.Flatten()(pretrained_model.output)

# Add a Dense layer with 128 units and ReLU activation
dense = layers.Dense(128, activation='relu')(flatten)

# Add the final output layer with 3 units and softmax activation
output = layers.Dense(3, activation='softmax')(dense)

# Create the model
model = Model(inputs=pretrained_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 262, 262,  │          0 │ input_layer_24[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 128, 128,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 128, 128,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 128, 128,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 130, 130,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 64, 64,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 64, 64,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 64, 64,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 64, 64,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 64, 64,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 64, 64,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 64, 64,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 64, 64,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 64, 64,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_3_c

 Total params: 23,850,371 (90.98 MB)

 Trainable params: 23,797,251 (90.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
resnet_model.compile(
    optimizer=Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = resnet_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

2/2 [==============================] - 2s 659ms/step - loss: 0.1941 - accuracy: 0.9730 - val_loss: 2.9398 - val_accuracy: 0.1111
